### 전체 지하철역 데이터 전처리(9호선은 없는듯 함)

In [376]:
import warnings
warnings.filterwarnings(action='ignore')
import scipy as sp
import scipy.stats
import statsmodels.api as sm
import sklearn as sk
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt
import glob
import pandas as pd
set(sorted([f.name for f in mpl.font_manager.fontManager.ttflist]))
mpl.rc('font', family='NanumGothic')
mpl.rc('axes', unicode_minus=False)

In [377]:
sub_2015 = pd.read_csv('2015sub.csv',encoding='euc-kr',thousands=',')
sub_2016 = pd.read_csv('2016sub.csv',encoding='euc-kr',thousands=',')
sub_2017 = pd.read_csv('2017sub.csv',encoding='euc-kr',thousands=',')
sub_2018 = pd.read_csv('2018sub.csv',encoding='euc-kr',thousands=',')
sub_2019 = pd.read_csv('2019sub.csv',encoding='euc-kr',thousands=',')


In [378]:
sub_2015.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2016.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2017.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2018.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2019.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']

In [379]:
sub=pd.concat([sub_2015,sub_2016,sub_2017,sub_2018,sub_2019])

In [380]:
sub

,날짜,역번호,역명,구분,7to8,8to9,9to10,17to18,18to19,19to20
0,2015-01-01,150,서울역(150),승차,572,1106,1474,3666,3738,3091
1,2015-01-01,151,시청(151),승차,104,127,233,1176,1133,850
2,2015-01-01,152,종각(152),승차,281,291,328,2176,2047,1767
3,2015-01-01,153,종로3가(153),승차,155,189,347,1954,1526,1297
4,2015-01-01,154,종로5가(154),승차,83,128,209,948,962,722
...,...,...,...,...,...,...,...,...,...,...
200799,2019-12-31,2825,신흥,하차,86,243,208,510,562,493
200800,2019-12-31,2826,수진,승차,542,813,436,436,395,245
200801,2019-12-31,2826,수진,하차,115,433,259,463,501,452
200802,2019-12-31,2827,모란,승차,342,482,253,375,418,242


In [381]:
# 행 개수 확인
len(sub_2015)+len(sub_2016)+len(sub_2017)+len(sub_2018)+len(sub_2019)

1004354

In [382]:
# index 재할당
sub.reset_index(inplace=True,drop=True)

In [383]:
# to_numeric
sub[['역번호', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']]=sub[['역번호', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']].apply(pd.to_numeric)

In [384]:
sub.날짜=pd.to_datetime(sub['날짜'])

In [385]:
sub.날짜=pd.to_datetime(sub['날짜'])
sub['year']=sub.날짜.apply(lambda x: x.year)
sub['month']=sub.날짜.apply(lambda x: x.month)

In [386]:
# 요일 찾기
import datetime
from datetime import date

t=['월','화','수','목','금','토','일']
sub["요일"]=sub['날짜'].apply(lambda x: t[x.weekday()])

In [387]:
sub

,날짜,역번호,역명,구분,7to8,8to9,9to10,17to18,18to19,19to20,year,month,요일
0,2015-01-01,150,서울역(150),승차,572,1106,1474,3666,3738,3091,2015,1,목
1,2015-01-01,151,시청(151),승차,104,127,233,1176,1133,850,2015,1,목
2,2015-01-01,152,종각(152),승차,281,291,328,2176,2047,1767,2015,1,목
3,2015-01-01,153,종로3가(153),승차,155,189,347,1954,1526,1297,2015,1,목
4,2015-01-01,154,종로5가(154),승차,83,128,209,948,962,722,2015,1,목
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004349,2019-12-31,2825,신흥,하차,86,243,208,510,562,493,2019,12,화
1004350,2019-12-31,2826,수진,승차,542,813,436,436,395,245,2019,12,화
1004351,2019-12-31,2826,수진,하차,115,433,259,463,501,452,2019,12,화
1004352,2019-12-31,2827,모란,승차,342,482,253,375,418,242,2019,12,화


```




```

### 2호선 퇴근시간만

In [388]:
# 2호선 퇴근시간만
sub2=sub[(200<sub["역번호"]) & (sub["역번호"]<300)].reset_index(drop=True)
sub2.drop(['7to8', '8to9', '9to10'],1,inplace=True)

In [389]:
sub2

,날짜,역번호,역명,구분,17to18,18to19,19to20,year,month,요일
0,2015-01-01,201,시청(201),승차,889,860,632,2015,1,목
1,2015-01-01,202,을지로입구(202),승차,2079,2456,2063,2015,1,목
2,2015-01-01,203,을지로3가(203),승차,450,371,329,2015,1,목
3,2015-01-01,204,을지로4가(204),승차,284,206,175,2015,1,목
4,2015-01-01,205,동대문역사문화공원(205),승차,1006,980,909,2015,1,목
...,...,...,...,...,...,...,...,...,...,...
182595,2019-12-31,248,양천구청,하차,761,989,843,2019,12,화
182596,2019-12-31,249,신정네거리,승차,501,544,343,2019,12,화
182597,2019-12-31,249,신정네거리,하차,1011,1454,881,2019,12,화
182598,2019-12-31,250,용두(동대문구청),승차,253,252,162,2019,12,화


In [390]:
# 17~20시간대의 탑승객
sub2["17to20"]=sub2["17to18"]+sub2["18to19"]+sub2["19to20"]

In [391]:
# 역명에서 역번호 제외
sub2["역명"]=sub2.역명.str.split("(",expand=True)[0]

In [392]:
sub2

,날짜,역번호,역명,구분,17to18,18to19,19to20,year,month,요일,17to20
0,2015-01-01,201,시청,승차,889,860,632,2015,1,목,2381
1,2015-01-01,202,을지로입구,승차,2079,2456,2063,2015,1,목,6598
2,2015-01-01,203,을지로3가,승차,450,371,329,2015,1,목,1150
3,2015-01-01,204,을지로4가,승차,284,206,175,2015,1,목,665
4,2015-01-01,205,동대문역사문화공원,승차,1006,980,909,2015,1,목,2895
...,...,...,...,...,...,...,...,...,...,...,...
182595,2019-12-31,248,양천구청,하차,761,989,843,2019,12,화,2593
182596,2019-12-31,249,신정네거리,승차,501,544,343,2019,12,화,1388
182597,2019-12-31,249,신정네거리,하차,1011,1454,881,2019,12,화,3346
182598,2019-12-31,250,용두,승차,253,252,162,2019,12,화,667


In [393]:
sub2.drop(["역번호","17to18","18to19","19to20"],1,inplace=True)

In [394]:
sub2

,날짜,역명,구분,year,month,요일,17to20
0,2015-01-01,시청,승차,2015,1,목,2381
1,2015-01-01,을지로입구,승차,2015,1,목,6598
2,2015-01-01,을지로3가,승차,2015,1,목,1150
3,2015-01-01,을지로4가,승차,2015,1,목,665
4,2015-01-01,동대문역사문화공원,승차,2015,1,목,2895
...,...,...,...,...,...,...,...
182595,2019-12-31,양천구청,하차,2019,12,화,2593
182596,2019-12-31,신정네거리,승차,2019,12,화,1388
182597,2019-12-31,신정네거리,하차,2019,12,화,3346
182598,2019-12-31,용두,승차,2019,12,화,667


In [395]:
len(sub2.역명.unique()),sub2.역명.unique()

(51,
 array(['시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리', '왕십리',
        '한양대', '뚝섬', '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '신천', '종합운동장',
        '삼성', '선릉', '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대', '서울대입구',
        '봉천', '신림', '신대방', '구로디지털단지', '대림', '신도림', '문래', '영등포구청', '당산',
        '합정', '홍대입구', '신촌', '이대', '아현', '충정로', '용답', '신답', '신설동', '도림천',
        '양천구청', '신정네거리', '용두', '잠실새내'], dtype=object))

In [396]:
# 신천이 잠실새내로 변경되어 두개의 이름 존재
sub2.loc[sub2['역명']=="신천",'역명']="잠실새내"

In [397]:
sub2

,날짜,역명,구분,year,month,요일,17to20
0,2015-01-01,시청,승차,2015,1,목,2381
1,2015-01-01,을지로입구,승차,2015,1,목,6598
2,2015-01-01,을지로3가,승차,2015,1,목,1150
3,2015-01-01,을지로4가,승차,2015,1,목,665
4,2015-01-01,동대문역사문화공원,승차,2015,1,목,2895
...,...,...,...,...,...,...,...
182595,2019-12-31,양천구청,하차,2019,12,화,2593
182596,2019-12-31,신정네거리,승차,2019,12,화,1388
182597,2019-12-31,신정네거리,하차,2019,12,화,3346
182598,2019-12-31,용두,승차,2019,12,화,667


In [398]:
len(sub2.역명.unique()),sub2.역명.unique()

(50,
 array(['시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리', '왕십리',
        '한양대', '뚝섬', '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '잠실새내',
        '종합운동장', '삼성', '선릉', '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대',
        '서울대입구', '봉천', '신림', '신대방', '구로디지털단지', '대림', '신도림', '문래', '영등포구청',
        '당산', '합정', '홍대입구', '신촌', '이대', '아현', '충정로', '용답', '신답', '신설동',
        '도림천', '양천구청', '신정네거리', '용두'], dtype=object))

In [399]:
np.sum(sub2.isnull())

날짜        0
역명        0
구분        0
year      0
month     0
요일        0
17to20    0
dtype: int64

### 2호선 데이터 공휴일 여부 표시

In [400]:
holi=pd.read_csv("holi.csv",encoding="euc-kr")

In [401]:
holi

,locdate,seq,isHoliday,dateName,year,week
0,20150101,1,Y,신정,2015,목
1,20150218,1,Y,설날,2015,수
2,20150219,1,Y,설날,2015,목
3,20150220,1,Y,설날,2015,금
4,20150301,1,Y,삼일절,2015,일
...,...,...,...,...,...,...
86,20190913,1,Y,추석,2019,금
87,20190914,1,Y,추석,2019,토
88,20191003,1,Y,개천절,2019,목
89,20191009,1,Y,한글날,2019,수


In [402]:
# seq 공휴일이 겹친경우 카운트 됨
holi[holi.seq==2].locdate

47    20171002
49    20171003
Name: locdate, dtype: int64

In [403]:
# 띠용 20171002는 seq 2 인데 한개 밖에 없눈데..
holi[(holi.locdate==20171002) | (holi.locdate==20171003)]

,locdate,seq,isHoliday,dateName,year,week
47,20171002,2,Y,임시공휴일,2017,월
48,20171003,1,Y,개천절,2017,화
49,20171003,2,Y,추석,2017,화


In [404]:
# isHoliday가 N인경우는 쉬는날이 아니었으므로 제거
holi=holi[holi.isHoliday!="N"]

In [405]:
holi.groupby("year").size()

year
2015    16
2016    17
2017    19
2018    18
2019    16
dtype: int64

In [406]:
holi.locdate=pd.to_datetime(holi.locdate,format="%Y%m%d")

In [407]:
# sub2.holi는 공휴일 -> 주말은 아님
sub2["holi"]=sub2["날짜"].isin(holi.locdate)

In [408]:
sub2[sub2["year"]==2017][sub2.holi==True]
# 1900개행이 아닌 1800개 --> 20171003이 개천절과 추석이 겹쳐있기 때문에 중복!

,날짜,역명,구분,year,month,요일,17to20,holi
73100,2017-01-01,시청,승차,2017,1,일,1901,True
73101,2017-01-01,시청,하차,2017,1,일,958,True
73102,2017-01-01,을지로입구,승차,2017,1,일,7001,True
73103,2017-01-01,을지로입구,하차,2017,1,일,4080,True
73104,2017-01-01,을지로3가,승차,2017,1,일,1772,True
...,...,...,...,...,...,...,...,...
108995,2017-12-25,양천구청,하차,2017,12,월,1073,True
108996,2017-12-25,신정네거리,승차,2017,12,월,861,True
108997,2017-12-25,신정네거리,하차,2017,12,월,1727,True
108998,2017-12-25,용두,승차,2017,12,월,264,True


In [409]:
import numpy as np
# 주말이거나 공휴일이면 holi가 True가 되도록 변경
sub2["holi"]=np.where((sub2["요일"].isin(["토","일"]))|(sub2["holi"]==True),True,False)

In [410]:
sub2[sub2.holi==True].head(60)

,날짜,역명,구분,year,month,요일,17to20,holi
0,2015-01-01,시청,승차,2015,1,목,2381,True
1,2015-01-01,을지로입구,승차,2015,1,목,6598,True
2,2015-01-01,을지로3가,승차,2015,1,목,1150,True
3,2015-01-01,을지로4가,승차,2015,1,목,665,True
4,2015-01-01,동대문역사문화공원,승차,2015,1,목,2895,True
5,2015-01-01,신당,승차,2015,1,목,1270,True
6,2015-01-01,상왕십리,승차,2015,1,목,649,True
7,2015-01-01,왕십리,승차,2015,1,목,2157,True
8,2015-01-01,한양대,승차,2015,1,목,698,True
9,2015-01-01,뚝섬,승차,2015,1,목,953,True


```
csv파일 생성 완료라 markdown으로 바꿔놓음
sub2.to_csv("sub2.csv",index=False,encoding="euc-kr")
```

## 신설동라인같은 역 제거 필요!!!

```





```

### 기상 데이터 전처리

In [411]:
ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 6850-4FA4

 C:\dss13\회귀분석 프로젝트 디렉터리

2020-06-12  오후 05:53    <DIR>          .
2020-06-12  오후 05:53    <DIR>          ..
2020-06-11  오후 08:14    <DIR>          .ipynb_checkpoints
2016-07-19  오후 02:05            86,180 2015day_1.csv
2020-06-11  오후 08:13        13,017,253 2015sub.csv
2017-03-22  오후 09:48            87,036 2016day_1.csv
2020-06-11  오후 08:13        13,055,897 2016sub.csv
2018-02-02  오후 12:38            87,067 2017day_1.csv
2020-06-11  오후 08:13        12,684,562 2017sub.csv
2019-02-03  오후 07:06            87,083 2018day_1.csv
2020-06-11  오후 08:13        11,462,196 2018sub.csv
2020-05-22  오전 10:15            87,003 2019day_1.csv
2020-06-11  오후 08:13        11,470,440 2019sub.csv
2020-06-11  오후 08:14             2,725 holi.csv
2020-06-12  오후 05:53           141,066 sub_data.ipynb
              12개 파일          62,268,508 바이트
               3개 디렉터리  26,861,641,728 바이트 남음


In [412]:
tem_2019= pd.read_csv('2019day_1.csv',encoding='euc-kr')
tem_2018= pd.read_csv('2018day_1.csv',encoding='euc-kr')
tem_2017= pd.read_csv('2017day_1.csv',encoding='euc-kr')
tem_2016= pd.read_csv('2016day_1.csv',encoding='euc-kr')
tem_2015= pd.read_csv('2015day_1.csv',encoding='euc-kr')


In [413]:
tem_2019.columns[36]

'합계 일사량(MJ/m2)'

In [414]:
tem_2019.rename(columns={'합계 일사량(MJ/m2)':'합계 일사(MJ/m2)'},inplace=True)

In [415]:
tem_2019.columns[36]

'합계 일사(MJ/m2)'

In [416]:
tem = pd.concat([tem_2019,tem_2018,tem_2017,tem_2016,tem_2015])

In [417]:
tem

,지점,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),강수 계속시간(hr),10분 최다 강수량(mm),10분 최다강수량 시각(hhmi),...,0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),안개 계속시간(hr),합계 일조 시간(hr)
0,108,2019-01-01,-5.0,-8.2,631,-0.6,1525.0,NaN,NaN,NaN,...,2.6,6.1,9.1,14.9,17.1,1.0,1.4,NaN,NaN,NaN
1,108,2019-01-02,-4.9,-8.8,802,0.2,1458.0,NaN,NaN,NaN,...,2.4,5.9,8.9,14.7,17.1,1.3,1.9,NaN,NaN,NaN
2,108,2019-01-03,-3.5,-8.4,726,3.2,1436.0,NaN,NaN,NaN,...,2.3,5.8,8.8,14.7,17.0,1.4,2.0,NaN,NaN,NaN
3,108,2019-01-04,-1.1,-6.2,757,4.1,1504.0,NaN,NaN,NaN,...,2.2,5.7,8.7,14.5,17.0,0.9,1.3,NaN,NaN,NaN
4,108,2019-01-05,-2.8,-5.5,2358,1.1,1521.0,NaN,NaN,NaN,...,2.2,5.5,8.6,14.5,16.9,1.4,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,108,2015-12-27,-5.7,-8.9,751,-1.0,1505.0,NaN,NaN,NaN,...,5.0,8.3,10.7,15.9,17.2,NaN,1.2,NaN,NaN,9.2
361,108,2015-12-28,-5.7,-9.5,631,-1.0,1558.0,NaN,NaN,NaN,...,4.4,8.1,10.6,15.8,17.2,NaN,1.6,NaN,NaN,9.2
362,108,2015-12-29,-3.2,-8.7,739,2.9,1549.0,NaN,NaN,NaN,...,4.0,7.9,10.5,15.8,17.2,NaN,1.3,NaN,NaN,7.6
363,108,2015-12-30,0.3,-3.5,608,4.3,1425.0,3.75,NaN,NaN,...,3.8,7.7,10.3,15.7,17.2,NaN,0.2,2.5,NaN,3.1


In [418]:
tem.reset_index(drop=True,inplace=True)

In [419]:
np.where(tem.columns.isin(['일시','평균기온(°C)','평균 상대습도(%)','일강수량(mm)','합계 일사(MJ/m2)']))

(array([ 1,  2, 12, 24, 36], dtype=int64),)

In [420]:
tem=tem.iloc[:,[1,2,12,24,36]]

In [421]:
tem.head()

,일시,평균기온(°C),일강수량(mm),평균 상대습도(%),합계 일사(MJ/m2)
0,2019-01-01,-5.0,NaN,49.5,7.84
1,2019-01-02,-4.9,NaN,42.8,10.48
2,2019-01-03,-3.5,NaN,38.8,10.28
3,2019-01-04,-1.1,NaN,55.5,6.20
4,2019-01-05,-2.8,NaN,40.3,10.05


In [422]:
tem.columns=["date","avgtemp","avgrainfall","avghimidity","solar"]

In [423]:
tem[tem.solar.isnull()]

,date,avgtemp,avgrainfall,avghimidity,solar
239,2019-08-28,26.1,NaN,66.2,NaN
240,2019-08-29,23.4,36.9,77.1,NaN
382,2018-01-18,2.1,0.0,59.6,NaN
390,2018-01-26,-14.8,NaN,34.9,NaN
410,2018-02-15,-0.7,NaN,28.0,NaN
835,2017-04-16,18.7,NaN,52.9,NaN
844,2017-04-25,16.0,NaN,56.8,NaN
1038,2017-11-05,8.4,NaN,52.5,NaN
1053,2017-11-20,-0.4,0.0,55.3,NaN
1060,2017-11-27,2.5,NaN,53.0,NaN


In [424]:
tem.date=pd.to_datetime(tem['date'])
tem['year']=tem.date.apply(lambda x: x.year)
tem['month']=tem.date.apply(lambda x: x.month)

In [425]:
a=np.where(tem.month==1)[0]

In [426]:
a[154]

1491

In [427]:
for i in range(len(a)):
    print(a[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491


In [428]:
tem.solar[239]

nan

In [429]:
msolar=tem.groupby(['year','month']).mean().reset_index()

In [430]:
a=[ 239,  240,  382,  390,  410,  835,  844, 1038, 1053, 1060, 1063,1615]
for i in a:
    tem.solar[i]=msolar.solar[(msolar.year==tem.year[i]) & (msolar.month==tem.month[i])]

In [431]:
tem.iloc[a,:]

,date,avgtemp,avgrainfall,avghimidity,solar,year,month
239,2019-08-28,26.1,NaN,66.2,16.766897,2019,8
240,2019-08-29,23.4,36.9,77.1,16.766897,2019,8
382,2018-01-18,2.1,0.0,59.6,5.482414,2018,1
390,2018-01-26,-14.8,NaN,34.9,5.482414,2018,1
410,2018-02-15,-0.7,NaN,28.0,11.454815,2018,2
835,2017-04-16,18.7,NaN,52.9,17.461071,2017,4
844,2017-04-25,16.0,NaN,56.8,17.461071,2017,4
1038,2017-11-05,8.4,NaN,52.5,5.590000,2017,11
1053,2017-11-20,-0.4,0.0,55.3,5.590000,2017,11
1060,2017-11-27,2.5,NaN,53.0,5.590000,2017,11


In [432]:
tem

,date,avgtemp,avgrainfall,avghimidity,solar,year,month
0,2019-01-01,-5.0,NaN,49.5,7.84,2019,1
1,2019-01-02,-4.9,NaN,42.8,10.48,2019,1
2,2019-01-03,-3.5,NaN,38.8,10.28,2019,1
3,2019-01-04,-1.1,NaN,55.5,6.20,2019,1
4,2019-01-05,-2.8,NaN,40.3,10.05,2019,1
...,...,...,...,...,...,...,...
1821,2015-12-27,-5.7,NaN,39.0,9.75,2015,12
1822,2015-12-28,-5.7,NaN,38.4,9.78,2015,12
1823,2015-12-29,-3.2,NaN,47.6,7.16,2015,12
1824,2015-12-30,0.3,2.5,67.6,5.43,2015,12


In [433]:
tem.avgrainfall=tem['avgrainfall'].fillna(0)

In [434]:
tem.tail(5)

,date,avgtemp,avgrainfall,avghimidity,solar,year,month
1821,2015-12-27,-5.7,0.0,39.0,9.75,2015,12
1822,2015-12-28,-5.7,0.0,38.4,9.78,2015,12
1823,2015-12-29,-3.2,0.0,47.6,7.16,2015,12
1824,2015-12-30,0.3,2.5,67.6,5.43,2015,12
1825,2015-12-31,0.7,0.0,81.3,3.98,2015,12
